# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sur,22.5667,59.5289,28.75,72,4,3.98,OM,1724357516
1,1,longyearbyen,78.2186,15.6401,11.91,71,75,4.63,SJ,1724357517
2,2,constantia,44.1833,28.6500,25.39,63,14,3.24,RO,1724357518
3,3,walvis bay,-22.9575,14.5053,15.67,62,0,3.60,NaN,1724357520
4,4,lebu,-37.6167,-73.6500,9.03,80,100,4.02,CL,1724357522


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles= "OSM",
    frame_height=400, 
    frame_width=600, 
    size='Humidity', 
    alpha=0.4, 
    color='City'
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = city_data_df['Max Temp'] < 30
humidity = city_data_df['Humidity'] < 60
wind_speed = city_data_df['Wind Speed'] < 3
combined_condition = max_temp & humidity & wind_speed
ideal_city_df = city_data_df[combined_condition]

# Drop any rows with null values
cleaned_df = ideal_city_df.dropna()

# Display sample data
cleaned_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,hodmezovasarhely,46.4167,20.3333,23.33,32,100,2.91,HU,1724357538
21,21,sao vicente,-23.9631,-46.3919,24.54,57,3,2.63,BR,1724357543
28,28,alice springs,-23.7000,133.8833,13.75,58,0,0.00,AU,1724357551
40,40,priladozhskiy,59.8500,31.4833,19.18,1,24,1.90,RU,1724357565
51,51,tiznit,29.5833,-9.5000,23.72,56,0,1.34,MA,1724357579


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =  cleaned_df.loc[:,["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
17,hodmezovasarhely,HU,46.4167,20.3333,32,
21,sao vicente,BR,-23.9631,-46.3919,57,
28,alice springs,AU,-23.7000,133.8833,58,
40,priladozhskiy,RU,59.8500,31.4833,1,
51,tiznit,MA,29.5833,-9.5000,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radiusMeters = 10000
params = {
     "categories" : "accommodation.hotel",
    "limit" : 10,
    "apiKey" : geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]    
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radiusMeters}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hodmezovasarhely - nearest hotel: Hotel Fáma
sao vicente - nearest hotel: Pousada Vitória
alice springs - nearest hotel: Aurora Alice Springs
priladozhskiy - nearest hotel: No hotel found
tiznit - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
semnan - nearest hotel: مرکز آموزشی رفاهی فرهنگیان - خانه معلم
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
penhalonga - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
lakeview - nearest hotel: No hotel found
sariyahsi - nearest hotel: No hotel found
gherla - nearest hotel: Pensiune Restaurant Cola
leh - nearest hotel: Kang Lha Chen
ust-nera - nearest hotel: гостиница "Солнечная"
anaconda - nearest hotel: Hickory House Inn
bukama - nearest hotel: Mission catholique
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
gilau - nearest hotel: Casa Zânelor
grants pass - nearest hotel: The Lodge
cocal - nearest hotel: Hotel Soratur
vernon - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
17,hodmezovasarhely,HU,46.4167,20.3333,32,Hotel Fáma
21,sao vicente,BR,-23.9631,-46.3919,57,Pousada Vitória
28,alice springs,AU,-23.7000,133.8833,58,Aurora Alice Springs
40,priladozhskiy,RU,59.8500,31.4833,1,No hotel found
51,tiznit,MA,29.5833,-9.5000,56,No hotel found
53,albany,US,42.6001,-73.9662,58,No hotel found
60,semnan,IR,35.5729,53.3971,23,مرکز آموزشی رفاهی فرهنگیان - خانه معلم
94,hotan,CN,37.0997,79.9269,50,Hua Yi International Hotel (accepts foreigners)
109,penhalonga,ZW,-18.8911,32.6978,51,No hotel found
115,jamestown,US,42.0970,-79.2353,45,DoubleTree Jamestown


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
hotel_data_map = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles= "OSM",
    frame_height=400, 
    frame_width=600, 
    size='Humidity', 
    alpha=0.4, 
    color='City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)